<a href="https://colab.research.google.com/github/z-gursky/Data-Cleaning-Candy-dataset-/blob/main/Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
%matplotlib inline
import io

In [2]:
# import my data 
from google.colab import files
data_to_load = files.upload()

Saving candy.csv to candy (12).csv


In [3]:
# Users may need to use utf-8 for encoding but ISO-8859-1 worked for me 
candy_df = pd.read_csv(io.BytesIO(data_to_load['candy.csv']), encoding = "ISO-8859-1")
print(candy_df.head())

   Internal ID Q1: GOING OUT?  ... Q12: MEDIA [Yahoo] Click Coordinates (x, y)
0     90258773            NaN  ...                NaN                      NaN
1     90272821             No  ...                NaN                 (84, 25)
2     90272829            NaN  ...                NaN                      NaN
3     90272840             No  ...                NaN                 (75, 23)
4     90272841             No  ...                NaN                 (70, 10)

[5 rows x 120 columns]


The dataset includes the following variables:

* **Internal ID:** A unique indentifier for every record in the database. 
* **Going Out?:** Binary field (Yes/No).
* **Gender:** Three different options (Male, Female, I'd rather not say).
* **Age:** Numerical field.
* **Country:** This is a text field but looks like people have entered their own versions --> USA instead of America.
* **State/Province:** Text field, again with users entering own versions. 
* **Joy Or Despair:** Three different options to rate candy (Joy, Meh, Despair).
* **Joy Other:** Text field.
* **Despair Other:** Text field.
* **Other Comments:** Text field.
* **Dress:** Text field.
* **Unnamed:**
* **Day:** Binary field.
* **Media:** Images/click fields. 

The goal of this dataset is for data cleaning. Starting off I will remove the column named "Unnamed: 113" as I do not know what the values are for (even though there are plenty of values in that column). Since the goal is for datacleaning, all other columns will be kept, even though in other scenarios when building models some may be dropped. 

In [4]:
# drop the columns we will not be working with
candy_df.drop(["Unnamed: 113"], axis=1, inplace=True)

Changing the column names to be all lower case to make the data easier to work with. If you would like you can also change the names of the columns as below, 

candy_df = candy_df.rename(columns = {'Q1: GOING OUT?' : 'going_out?', 'Q2: GENDER' : 'gender'}) and so on. 

In [5]:
candy_df.columns = candy_df.columns.str.lower()

Lets take a look at missing values with percentage. 

In [6]:
def missing_data(df):
  df = df.isnull().sum()*100/df.isnull().count()
  print(df)
missing_data(candy_df)

internal id                  0.000000
q1: going out?               4.471545
q2: gender                   1.666667
q3: age                      3.414634
q4: country                  2.601626
                              ...    
q12: media [daily dish]     96.544715
q12: media [science]        44.634146
q12: media [espn]           95.975610
q12: media [yahoo]          97.276423
click coordinates (x, y)    34.756098
Length: 119, dtype: float64


By looking at the above we can tell that Media [Yahoo], Media[ESPN], Media[Daily Dish], Despair Other, and Other Comments are columns with more than 70% of missing values. I will go ahead and drop these columns by using the same drop method above.

In [7]:
# drop the columns we will not be working with
candy_df.drop(['q8: despair other','q9: other comments','q12: media [yahoo]', 
               'q12: media [daily dish]','q12: media [espn]', 'click coordinates (x, y)'], axis=1, inplace=True)

Now lets look at other fields and figure out how to handle all other missing values. 

In [8]:
print(candy_df['q2: gender'].isna().value_counts())

False    2419
True       41
Name: q2: gender, dtype: int64


The above tells us that there are 41 missing values with the Gender column. This field is not just "Male" or "Female, it also includes "I'd rather not say" and "Other". Because the field is left open, the person filling out the form may have not wanted to give this information so I will fill the values with "I'd rather not say". 

In [9]:
candy_df['q2: gender'].fillna(value="I'd rather not say", inplace=True)

The country column is not a pretty one. The form is completed by writing in a response so the answers vary. To make sure I do not mess up the dataframe when figuring out the best way to move forward I will first make a copy of the dataframe. 

When using a form and data manually being entered it can lead to errors. These errors could be misspelling, entering mis-information, capital letters where they should'nt be, and white spaces. Lets start with fixing white spaces and capitalization and see if that does any good. 

In [10]:
candy_df['q4: country'] = candy_df['q4: country'].str.lower()
candy_df['q4: country'] = candy_df['q4: country'].str.strip()
missing_data(candy_df)

internal id                      0.000000
q1: going out?                   4.471545
q2: gender                       0.000000
q3: age                          3.414634
q4: country                      2.601626
                                  ...    
q6 | york peppermint patties    28.658537
q7: joy other                   62.967480
q10: dress                      30.243902
q11: day                        29.471545
q12: media [science]            44.634146
Length: 113, dtype: float64


Stripping the data of white spaces did nothing. Let's use unique() which will list all unique values in order of appererance. This will give us a good representation of what we're dealing with. 

In [11]:
candy_df["q4: country"].unique()

array([nan, 'usa', 'us', 'canada', 'murica', 'united states', 'uk',
       'united kingdom', 'united staes', 'united states of america',
       'uae', 'england', 'mexico', 'u.s.a.', 'usausausa', 'america', '35',
       'france', 'finland', 'unhinged states', 'us of a', 'unites states',
       'the united states', 'north carolina', 'unied states',
       'netherlands', 'germany', 'europe', 'earth', 'u s', 'u.s.', 'u.k.',
       'costa rica', 'the united states of america', 'unite states', '46',
       'cascadia', 'australia', 'insanity lately', 'greece',
       'usa? hard to tell anymore..', "'merica", 'usas', 'pittsburgh',
       '45', 'united state', '32', 'a', 'can', 'canae', 'new york',
       'trumpistan', 'ireland', 'united sates', 'korea', 'california',
       'japan', 'south africa',
       'i pretend to be from canada, but i am really from the united states.',
       'iceland', 'canada`', 'scotland', 'denmark', 'united stated',
       'switzerland', 'ahem....amerca', 'ud', 'sou

There are a lot of different ways to deal with this data. Using regex to find patterns or replace words. We could use fuzzywuzzy a python method to match strings looks at differences in sequences. I'll be using replace which you can see below. 

In [12]:
candy_df["q4: country"] = candy_df["q4: country"].replace(['usa', 'us', 
        'murica', 'united states', 'united staes', 'united states of america',
       'uae', 'u.s.a.', 'usausausa', 'america', '35', 'unhinged states', 'us of a', 'unites states',
       'the united states', 'north carolina', 'unied states', 'earth', 'u s', 'u.s.', 
       'costa rica', 'the united states of america', 'unite states', '46', 'usa? hard to tell anymore..', "'merica", 'usas', 'pittsburgh',
       '45', 'united state', '32', 'a', 'new york', 'united sates', 'california', 
       'i pretend to be from canada, but i am really from the united states.', 
       'united stated', 'ahem....amerca', 'new jersey', 'united ststes','united statss',
       'atlantis', 'murrika', 'usa! usa! usa!', 'usaa',
       'alaska', 'n. america', 'ussa', 'narnia',
       'u s a', 'united statea', '1', 'subscribe to dm4uz3 on youtube',
       'usa usa usa!!!!', "i don't know anymore", 'fear and loathing', 'insanity lately'], "usa")

In [13]:
candy_df["q4: country"].unique()

array([nan, 'usa', 'canada', 'uk', 'united kingdom', 'england', 'mexico',
       'france', 'finland', 'netherlands', 'germany', 'europe', 'u.k.',
       'cascadia', 'australia', 'greece', 'can', 'canae', 'trumpistan',
       'ireland', 'korea', 'japan', 'south africa', 'iceland', 'canada`',
       'scotland', 'denmark', 'switzerland', 'ud', 'south korea',
       'indonesia', 'the netherlands', 'endland', 'soviet canuckistan',
       'singapore', 'china', 'taiwan', 'hong kong', 'spain', 'sweden'],
      dtype=object)

Seemed to clean up a lot of the country column. Now that we have what we want I'll use mode to replace any missing data. You could also replace missing data with "Do not know" or anything that makes sense. 

In [14]:
candy_df["q4: country"].fillna(candy_df["q4: country"].mode()[0], inplace=True)

In [15]:
# check to see the missing data in country column
missing_data(candy_df)

internal id                      0.000000
q1: going out?                   4.471545
q2: gender                       0.000000
q3: age                          3.414634
q4: country                      0.000000
                                  ...    
q6 | york peppermint patties    28.658537
q7: joy other                   62.967480
q10: dress                      30.243902
q11: day                        29.471545
q12: media [science]            44.634146
Length: 113, dtype: float64


Moving on to age column I'll be using the code below to check the different problems that occur within the column. 

In [16]:
# Print all values that cannot be converted to float
for column_name in ["q3: age"]:
  print("These are the problematic values for the variable: {}".format(column_name))
  lst = []
  for value in candy_df[column_name]:
    try:
      float(value)
    except:
      print(value)
      lst.append(value)

These are the problematic values for the variable: q3: age
Old enough
Many
?
no
45-55
hahahahaha
older than dirt
5u
Enough
See question 2
24-50
Over 50
sixty-nine
46 Halloweens.
ancient
OLD
old
old
70 1/2
MY NAME JEFF
59 on the day after Halloween
old enough
your mom
I can remember when Java was a cool new language
60+


People are creative! Again I'll be replacing any entry that doesnt make sense with np.nan and then we need to downcast to a float and use the mean to fill. 

In [17]:
candy_df = candy_df.replace(["Old enough", "?", "Many", "no", "45-55", "hahahahaha",
                             "older than dirt", "5u", "Enough", "See question 2",
                             "24-50", "Over 50", "sixty-nine", "46 Halloweens.",
                             "ancient", "OLD", "old", "70 1/2", "MY NAME JEFF",
                             "59 on the day after Halloween", "old enough", "your mom",
                             "I can remember when Java was a cool new language", "60+"], np.nan)

In [18]:
# Print all values that cannot be converted to float
for column_name in ["q3: age"]:
  print("These are the problematic values for the variable: {}".format(column_name))
  lst = []
  for value in candy_df[column_name]:
    try:
      float(value)
    except:
      print(value)
      lst.append(value)

These are the problematic values for the variable: q3: age


In [19]:
candy_df["q3: age"] = pd.to_numeric(candy_df["q3: age"], downcast="float")
candy_df["q3: age"].fillna(candy_df["q3: age"].mean(), inplace=True)

In [20]:
# check age column for missing data 
missing_data(candy_df)

internal id                      0.000000
q1: going out?                   4.471545
q2: gender                       0.000000
q3: age                          0.000000
q4: country                      0.000000
                                  ...    
q6 | york peppermint patties    28.658537
q7: joy other                   62.967480
q10: dress                      30.243902
q11: day                        29.471545
q12: media [science]            44.634146
Length: 113, dtype: float64


Going out column has some missing data as well. This entry should only accept Yes/No but for any missing columns here I will change to "Not sure" 

In [21]:
candy_df["q1: going out?"] = candy_df["q1: going out?"].fillna("Not sure")

If we have a closer look at the dataframe, we have a lot of missing data still. With an even closer look we can see that a lot of people filling out this survey left all answers empty when rating candy (which should accept three entries, Joy, Meh, Despair). Depending on what we're using the data for, that might be the primary focus. Due to this, below I will be dropping any row that does not have more than two candy rating questions filled out on their survey. 

In [22]:
# make a list of all column names 
my_list = candy_df.columns.values.tolist()

In [23]:
# initiate a new list and slice on the columns we need (candy ratings)
cols = my_list[6:-3]

In [24]:
len(cols)

104

At the moment, after all the cleaning we've done we're left with 2460 rows and 113 columns. Lets see what happens after we drop rows that do not have more than two candy ratings filled out. 

In [25]:
candy_df.shape

(2460, 113)

We use cols as our column list and set the thresh at 2. 

In [28]:
candy_df = candy_df.dropna(subset=cols, thresh=2)

In [29]:
candy_df.shape

(1803, 113)

We dropped a significant amount of rows and our missing data improved greatly. 

In [31]:
missing_data(candy_df)

internal id                      0.000000
q1: going out?                   0.000000
q2: gender                       0.000000
q3: age                          0.000000
q4: country                      0.000000
                                  ...    
q6 | york peppermint patties     2.662230
q7: joy other                   49.473100
q10: dress                       4.991681
q11: day                         3.882418
q12: media [science]            24.625624
Length: 113, dtype: float64


This is a great start and will continue to add code and update as time allows. 